In [3]:
import librosa
import IPython.display as ipd
from librosa.feature.spectral import spectral_centroid
import sklearn
import numpy as np
import soundfile

In [154]:
# load mel
mel = np.load("C:\\Users\\HP\\Desktop\\mtvs-ai\\9월프젝\\arena_mel\\0\\0.npy")
sr =16000
#hop_length = int( np.ceil(sr*0.01) )
#win_length = int( np.ceil(sr*0.025) )
#n_fft = int( np.ceil(sr*0.025) )
frame_length=512
hop_length = 256
win_length = 512
n_fft =512
pad_mode = 'reflect'
window='hann'
power=2.0
n_iter=32
norm='slaney'
#numbands=48, sampleRate=16000, frameSize=512, hopSize=256, window='hann', weighting='linear', warpingFormula='slaneyMel', normalize='unit_tri'


In [172]:
def music_features (song_id, song_name, genre, mel):
    attr = {'song id' : song_id, 'song name' : song_name, 'genre' : genre}
    
    stft =librosa.feature.inverse.mel_to_stft(mel, sr=sr, n_fft=n_fft, power=power, norm=norm)
    ms = np.abs(stft)## magnitude spectrogram
    ps = np.power(stft,2) ## power spectrogram

    chromagram =librosa.feature.chroma_stft(y=None, sr=sr, S=ps, norm=np.inf, n_fft=n_fft, 
                                            hop_length=hop_length, win_length=win_length,
                                            window=window, center=True, pad_mode=pad_mode,
                                            tuning=None, n_chroma=12)
    chroma_stft_mean = chromagram.mean()
    chroma_stft_var = chromagram.var()
    attr['chroma_stft_mean'] = chroma_stft_mean
    attr['chroma_stft_var'] = chroma_stft_var

    rms = librosa.feature.rms(y=None, S=ms, frame_length=frame_length, hop_length=hop_length, center=True, pad_mode=pad_mode)
    rms_mean = rms.mean()
    rms_var = rms.var()
    attr['rms_mean'] = rms_mean
    attr['rms_var'] = rms_var

    spectral_centroids = librosa.feature.spectral_centroid(y=None, sr=sr, S=ms, 
                                                           n_fft=n_fft, hop_length=hop_length, freq=None, 
                                                           win_length=win_length, window=window, 
                                                           center=True, pad_mode=pad_mode)
    spectral_centroid_mean = spectral_centroids.mean()
    spectral_centroid_var = spectral_centroids.var()
    attr['spectral_centroid_mean'] = spectral_centroid_mean
    attr['spectral_centroid_var'] = spectral_centroid_var
    
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=None, sr=sr, S=ms, n_fft=n_fft, 
                                                            hop_length=hop_length, win_length=win_length, window=window, 
                                                            center=True, pad_mode=pad_mode, freq=None, 
                                                            centroid= spectral_centroids, norm=True, p=power)
    spectral_bandwidth_mean = spectral_bandwidth.mean()
    spectral_bandwidth_var = spectral_bandwidth.var()
    attr['spectral_bandwidth_mean'] = spectral_bandwidth_mean
    attr['spectral_bandwidth_var'] = spectral_bandwidth_var

    spectral_rolloff = librosa.feature.spectral_rolloff(y=None, sr=sr, S=ms, n_fft=n_fft, 
                                                        hop_length=hop_length, win_length=win_length, 
                                                        window=window, center=True, pad_mode=pad_mode, 
                                                        freq=None, roll_percent=0.85)
    rolloff_mean = spectral_rolloff.mean()
    rolloff_var = spectral_rolloff.var()
    attr['rolloff_mean'] = spectral_bandwidth_mean
    attr['rolloff_var'] = spectral_bandwidth_var
    
        
    mfcc_s = librosa.feature.mfcc(y=None, sr=sr, S=mel, n_mfcc=20, dct_type=2, lifter=40)
    i = 0
    for mfcc in mfcc_s:
        attr['mfcc_mean_{}'.format(i)] = globals()['mfcc_mean_{}'.format(i)] = mfcc.mean()
        attr['mfcc_var_{}'.format(i)] = globals()['mfcc_var_{}'.format(i)] = mfcc.var()
        i+=1

 
    
    
    onset_envelope = librosa.onset.onset_strength(y=None, sr=sr, S=mel, lag=1, max_size=1, 
                                 ref=None, detrend=False, center=True,
                                 feature=None, aggregate=None)

#########rhythm features#########################################################################
    
    tempo_gram = librosa.feature.fourier_tempogram(y=None, sr=sr, onset_envelope=onset_envelope,
                                                   hop_length=hop_length, win_length=win_length,
                                                   center=True, window=window)
    tempogram_mean = tempo_gram.mean()
    tempogram_var = tempo_gram.var()
    attr['tempogram_mean'] = tempogram_mean
    attr['tempogram_var'] = tempogram_var
    
    return attr 

In [61]:
import pandas as pd

song_genre_df = pd.read_csv("C:\\Users\\HP\\Desktop\\mtvs-ai\\9월프젝\\csv\\song_genre_final_final.csv")

In [173]:
attr = music_features(song_genre_df['id'][0],
                       song_genre_df['song_name'][0],
                       song_genre_df['장르'][0],
                       mel)
df = pd.DataFrame()
df = df.append(attr, ignore_index=True)
df

,song id,song name,genre,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,...,mfcc_mean_16,mfcc_var_16,mfcc_mean_17,mfcc_var_17,mfcc_mean_18,mfcc_var_18,mfcc_mean_19,mfcc_var_19,tempogram_mean,tempogram_var
0,0.0,Feelings,POP,0.345399,0.126025,0.04889,0.002256,198.004306,55194.982338,95.837523,...,-72.842896,13304.270508,-36.774445,16541.097656,-82.672569,13456.097656,17.78616,15425.751953,0.569983+0.003354j,688.536682


chroma_stft_mean 
chroma_stft_var 
-> power spectrogram 필요
librosa.core.amplitude_to_db(S, ref=1.0, amin=1e-05, top_db=80.0)
librosa.core.db_to_power(S_db, ref=1.0)

In [ ]:
# 결과값 pandas dataframe 생성

In [52]:
import os
import pandas as pd

melon_features = pd.DataFrame() # 데이터 프레임 선언
idx = 0
dir_path = "C:\\Users\\HP\\Desktop\\mtvs-ai\\9월프젝\\arena_mel\\"
dir_list = os.listdir(dir_path)
for d in dir_list:
    files_path = os.path.join(dir_path, d + '\\')
    for f in os.listdir(files_path):
        
        f_path = os.path.join(files_path,f)
        mel_spectrogram = np.load(f_path)
        
        features_dic = music_features(song_genre_df['id'][idx],
                       song_genre_df['song_name'][idx],
                       song_genre_df['장르'][idx],
                       mel_spectrogram)
        
        melon_features = melon_features.append(features_dic, ignore_index=True)
        idx +=1
     


KeyboardInterrupt: 

In [ ]:
# 데이터 프레임 csv로 저장

## 코사인 유사도 구하기

In [ ]:
# csv load

df_30 = pd.read_csv('./features_30_sec_.csv', index_col = 'filename')

In [ ]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis = axis)

df_30 = df_30.drop(columns=['length', 'label'])
df_30_scaled = sklearn.preprocessing.scale(df_30)
sim_df = pd.DataFrame(df_30_scaled, columns=df_30.columns)
sim_df.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(df_30) # 모든 파일 과 목소리 하나의 코사인 유사도 구하기

music = sim_df['songfile_name'].sort_values(ascending = False)
music.index[1:5] # [0]은 자기 자신이므로 제외